## Feature embeddings / feature learning

Figuring out which features to put where is a large part of creating decent models.

## One-hot encoding

![](onehot.png)

When you have large vocabularies, one-hot encoding uses many dimensions.

## Word embeddings

* Word embeddings encodes 'concepts' into vector values.
* Word embeddings are learned
  * For instance from an autoencoder or dimensionality reduction
* Normally around 256 - 1024 features

![](embedding.png)

## Creating word embeddings with Keras

     Word index  ------> Embedding layer  -------> Word vector
   
   
Here we encode a vocabulary of 10'000 into a 64 dimensional embedding
```python
from keras.layers import Embedding

embedding_layer = Embedding(10000, 64)
```

In [ ]:
from keras.datasets import imdb

# Number of words to consider as features
max_features = 10000

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [ ]:
x_train[:2]

In [14]:
y_train[:2]

array([1, 0])

In [13]:
from keras import preprocessing

# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
model.add(Embedding(10000, 8, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

In [8]:
model.predict(x_train[:1])

array([[0.8316831]], dtype=float32)

## Exercise: Train word embedding on sklearn data

https://keras.io/datasets/ has a dataset on 11,228 newswires from Reuters, labeled over 46 topics.
Instead of classifying a binary IMDB review, you have to classify the text into one of the 46 topics.

1. Create a `Sequential` model that encodes the newswires into an embedding dimension of 256
2. Add layers to your neural network
  * Be creative; which layers do you think could help classifying this?
3. Compile, train and test the model

## Transfer learning

Transfer learning takes the **representation of a NN layer and reuses it to something else**.

See https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e

![](cnn.png)

![](cnn2.png)

![](cnn3.png)

Generally, the initial layers process basic features. The higher layers process more abstract things like leaves, noses, movements. **Just like in humans**.

## Advantages of transfer learning

1. We can re-use another training set
  * Or reduce the amount of training data needed
2. We save computational power is required
  * We are using pre-trained weights and only have to learn the weights of the last few layers.

## Pretrained models

Many pretrained models exist, which have been trained and optimised on huge datasets.

https://keras.io/applications/

## Example: Transfer learning with MobileNet

Example from https://github.com/aditya9898/transfer-learning

In [ ]:
!ls train

In [ ]:
!ls train/dogs

In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
#imports the mobilenet model and discards the last 1000 neuron layer
base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)

# add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) 
x = Dense(1024,activation='relu')(x) # dense layer 2
x = Dense(512,activation='relu')(x)  # dense layer 3
preds = Dense(3,activation='softmax')(x) # final layer with softmax activation

In [ ]:
model = Model(inputs=base_model.input, outputs=preds)

In [18]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
# Preprocessor for images
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 

# this is where you specify the path to the main data folder
train_generator = train_datagen.flow_from_directory('./train/', target_size=(224,224), color_mode='rgb',
                                                 batch_size=32, class_mode='categorical', shuffle=True)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train, 
                    epochs=5)

## Recap

* Transfer learning: taking one previously trained model and reusing it
* Closely related to multitask learning, where the *same model is used to solve different problems*
  * Forces the model to converge on a *common* representation
* This works because the neural network networks are *abstractions* of the things you are training
  * If abstract enough, you can use this for many other tasks